In [1]:

import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

ohlcv

serv = ServeNewOHLCV(ohlcv)
# serv.serv_period(date='01-01-2024', start_time='09:30:00', end_time='16:00:00')
# serv.serv_period(days_ago=3, months_ago=0, start_time='08:30:00', end_time='18:00:00')
start_data = serv.serv_range(dayrange=(-5,-2), start_time='08:30:00', end_time='18:00:00')
start_data


,open,high,low,close,volume
date,,,,,
2024-04-09 08:45:00,106.84,107.45,106.83,107.22,5564
2024-04-09 09:00:00,106.80,107.17,106.65,107.02,500
2024-04-09 09:15:00,107.31,107.53,106.86,106.99,955
2024-04-09 09:30:00,107.22,107.71,107.08,107.39,500
2024-04-09 09:45:00,107.11,107.50,107.06,107.50,500
...,...,...,...,...,...
2024-04-12 17:00:00,110.32,110.39,109.89,109.99,7768
2024-04-12 17:15:00,110.07,110.17,109.34,109.64,500
2024-04-12 17:30:00,109.54,109.94,109.29,109.66,7626


In [2]:
serv.next_bar(histBars=1)

,open,high,low,close,volume
date,,,,,
2024-04-12 18:00:00,109.61,109.72,109.32,109.58,500
2024-04-12 18:15:00,110.00,110.07,109.41,109.58,500


In [3]:
# serv.serv_range(dayrange=(2,8), start_time='08:30:00', end_time='18:00:00')

In [4]:
from frame.frame import Frame
f = Frame('TSLA')

#* KEEP RUNNING THIS CELL TO SEE THE CHART UPDATE

serv.next_bar(histBars=1)
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.plot(trading_hours=True)

In [5]:

from strategies.ta import MA, MACD

f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'red', 'width': 2}) # creates new column to the dataframe with the moving average eg MA_C9
f.add_ta(MA('close', 21), {'dash': 'solid', 'color': 'blue', 'width': 2})
f.add_ta(MA('close', 28), {'dash': 'solid', 'color': 'green', 'width': 2})
f.add_ta(MACD(fast=12, slow=26, signal=9) , 
         [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
          {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
          {'color': 'black'}], 'macd', 2) # histogram

f.update_ta_data()
f.plot(trading_hours=True)

In [6]:

from dataclasses import dataclass

@dataclass
class Strategy:
    data_serv: ServeNewOHLCV

    def __post_init__(self):
        self.f = Frame('TSLA')
        self.f.load_ohlcv(self.data)
        self.f.setup_chart()
        self.f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'red', 'width': 2})

In [7]:
f.load_ohlcv(serv.next_bar(histBars=1))
f.data.tail(12)

                       open    high     low   close  volume
date                                                       
2024-04-12 18:30:00  109.91  110.54  109.59  110.34    6035
2024-04-12 18:45:00  110.42  110.86  110.19  110.57     500


,open,high,low,close,volume,MA_cl_9,MA_cl_21,MA_cl_28,MACD_cl_12_26_9_MACD,MACD_cl_12_26_9_Signal,MACD_cl_12_26_9_Histogram
date,,,,,,,,,,,
2024-04-12 16:00:00,110.39,110.65,110.04,110.32,29289,109.852222,109.839524,110.016429,-0.062256,-0.160384,0.098128
2024-04-12 16:15:00,110.43,110.73,110.24,110.28,500,109.946667,109.804286,110.031429,-0.026976,-0.133702,0.106726
2024-04-12 16:30:00,109.91,110.58,109.83,110.33,21827,110.010000,109.808571,110.032500,0.004961,-0.105969,0.110930
2024-04-12 16:45:00,110.23,110.71,109.91,110.57,500,110.105556,109.834286,110.053571,0.049071,-0.074961,0.124033
2024-04-12 17:00:00,110.32,110.39,109.89,109.99,7768,110.128889,109.849048,110.037500,0.036804,-0.052608,0.089412
2024-04-12 17:15:00,110.07,110.17,109.34,109.64,500,110.103333,109.830000,109.993214,-0.001147,-0.042316,0.041169
2024-04-12 17:30:00,109.54,109.94,109.29,109.66,7626,110.081111,109.816190,109.931071,-0.029272,-0.039707,0.010435
2024-04-12 17:45:00,109.43,109.74,109.25,109.71,500,110.077778,109.796190,109.886071,-0.046986,-0.041163,-0.005823
2024-04-12 18:00:00,109.61,109.72,109.32,109.58,500,110.008889,109.787143,109.834643,-0.070698,-0.047070,-0.023628


# Putting it all together

In [32]:
import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV
from strategies.ta import MA, MACD, HPLP, SupRes, ATR
from frame.frame import Frame


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

serv = ServeNewOHLCV(ohlcv)
start_data = serv.serv_range(dayrange=(-5,-2), start_time='08:30:00', end_time='18:00:00') # used as a starting point for the backtest


f = Frame('TSLA')
f.add_ta(ATR(span=50), {}, chart_type=None)
f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'magenta', 'width': 2}) # appends ta to the ta list
f.add_ta(MA('close', 21), {'dash': 'solid', 'color': 'blue', 'width': 2})
f.add_ta(MA('close', 28), {'dash': 'solid', 'color': 'green', 'width': 2})
f.add_ta(MACD(fast=12, slow=26, signal=9) , 
         [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
          {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
          {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high
          {'color': 'red', 'size': 10}], 
          chart_type = 'points')

f.add_ta(SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance') 
# f.add_ta(SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', pointsAgo=2, tolerance=0.5),
#             [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
#             {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
#             chart_type = 'support_resistance') 


serv.next_bar(histBars=1) 
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.update_ta_data()
f.plot(trading_hours=True, height=1200, width=2000)


# backtest = BacktestFrames(f, serv, strategies=[strat1], barsToRun=20, stopAtEndOfDay=True)

"""
Resistance: 

The objective is to modify a support and resistance class that will find the support and resistance levels and return a pandas data frame with those levels projected forward to the end of the data frame from the points that have been detected
So it is important that every row of the data frame is filled in with the same price point stretching from the index where the point is located to the end of the data frame so that it displays correctly on a chart. (This is not a visualisation task but a data manipulation task)
Do not worry about tolerances and touch points for now.

  -- All code snippets are to be returned in the chat do not update files directly
  -- compute only base on the last close price on the last bar
  -- hp and lp is high point or low point 
  -- HP and LP are already computed and should be obtained from the data frame by referencing the relevant columns
  -- find the next  hp or lp That is higher than the close price. I suggest putting the mall in a list both high points and low points and sorting them in ascending order first.
  -- When finding the next higher point use the argument pointsAgo to find the point that is x points higher from the current point.
  -- now find to index Of the point that was found whether it was a low point or a high point is irrelevant. 
  -- Now project the point that was found and forward fill it to the end of the df So that when visualised on a chart there will be a line stretching from the point that has been found to the end of the chart. 
  
  Support:
  -- Similar to resistance but of course the opposite.

  """

# backtest.run()

'\nResistance: \n\nThe objective is to modify a support and resistance class that will find the support and resistance levels and return a pandas data frame with those levels projected forward to the end of the data frame from the points that have been detected\nSo it is important that every row of the data frame is filled in with the same price point stretching from the index where the point is located to the end of the data frame so that it displays correctly on a chart. (This is not a visualisation task but a data manipulation task)\nDo not worry about tolerances and touch points for now.\n\n  -- All code snippets are to be returned in the chat do not update files directly\n  -- compute only base on the last close price on the last bar\n  -- hp and lp is high point or low point \n  -- HP and LP are already computed and should be obtained from the data frame by referencing the relevant columns\n  -- find the next  hp or lp That is higher than the close price. I suggest putting the ma

In [2]:
style  = {'dash': 'solid', 'color': 'green', 'fillcolour': 'rgb(0, 255, 61, 0.1)', 'width': 2}
style  = {'dash': 'solid', 'color': 'green',  'width': 2}
fillcolour = style.get('fillcolour', 'rgba(0, 0, 255, 0.1)')

fillcolour

'rgba(0, 0, 255, 0.1)'

In [2]:
f.data.tail(12)

,open,high,low,close,volume,ATR_14,MA_cl_9,MA_cl_21,MA_cl_28,MACD_cl_12_26_9_MACD,...,Res_1_Lower,Res_2,Res_2_Upper,Res_2_Lower,Sup_1,Sup_1_Upper,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower
date,,,,,,,,,,,,,,,,,,,,,
2024-04-12 15:30:00,69.18,69.53,69.01,69.42,50000,0.347143,69.658889,70.004762,70.148571,-0.242913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 15:45:00,69.70,69.93,69.61,69.87,500,0.362857,69.643333,69.998571,70.115714,-0.212758,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 16:00:00,69.71,70.03,69.66,69.90,5726,0.375714,69.613333,69.976667,70.069286,-0.184315,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 16:15:00,69.88,70.01,69.73,69.97,500,0.370714,69.625556,69.960476,70.031429,-0.154346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 16:30:00,69.78,69.97,69.76,69.87,3000,0.365714,69.648889,69.949048,70.000000,-0.137084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 16:45:00,69.65,69.78,69.43,69.49,20543,0.359286,69.635556,69.925238,69.973571,-0.152311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 17:00:00,69.56,69.73,69.13,69.30,500,0.362857,69.591111,69.873810,69.946786,-0.177662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 17:15:00,69.53,69.68,69.39,69.40,500,0.362857,69.613333,69.827619,69.925000,-0.187522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 17:30:00,69.33,69.50,69.10,69.21,500,0.380714,69.603333,69.771905,69.896786,-0.208266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# create a df with random hp and lp points and random intervals between them. the points should not be on every row.  the y only occure evry so often  and the intervals between them are random. us np.nan to fill in the gaps

import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV
from strategies.ta import MA, MACD, HPLP, SupRes
from frame.frame import Frame


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

serv = ServeNewOHLCV(ohlcv)
start_data = serv.serv_range(dayrange=(-5,-2), start_time='08:30:00', end_time='18:00:00') # used as a starting point for the backtest
f = Frame('TSLA')

f.add_ta(HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high
          {'color': 'red', 'size': 10}], 
          chart_type = 'points')
serv.next_bar(histBars=1) 
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.update_ta_data()
f.plot( height=800, width=1200)

hplp = f.data[['close', 'HP_hi_10', 'LP_lo_10']].copy() # ['close', 'HP', 'LP']


In [53]:

# 1. get close price from the last bar 
# 2. get all high points and low points in alist and sort them in ascending order
# 3. if the last bar is green ie bullish then finf resistnce if bearish find support
# 4. assuming a last bullish bar  find the next point that is higher than the close price
# 5. find the upper and lower bounds by seeing the max and min of the high and low points within a renge as determineed by a tolerance value which then computes the ART multiple of the range
# log resistance and its bounds in a dictionary
# now do the same agians for the next resisatne level using the last resistance level upperbounds (plus a tolerance factor) as the starting point instead fo the close price
#  now do the same for the support levels but in the opposite direction but start with the min of the  lowerbounds of the low values and the lowerbouns of the res  as the starting point

hplp.rename(columns={'HP_hi_10': 'HP', 'LP_lo_10': 'LP'}, inplace=True)


# 1. get close price from the last bar 
close_price = f.data['close'].iloc[-1]

# 2. get all high points and low points in alist and sort them in ascending order
hp = hplp['HP'].dropna().to_list()
lp = hplp['LP'].dropna().to_list()
hp_lp = hp + lp
print(f'{hp_lp=}')
print('------')


# 3. if the last bar is green ie bullish then finf resistnce if bearish find support
last_close = f.data['close'].iloc[-1]
last_open = f.data['open'].iloc[-1]
last_color = 'green' if last_close > last_open else 'red'
print(f'{last_color=}')
print(f'{last_close=}')
print('------')

# 4. function to find the next higher or lower val in a list 
def find_next_val(val, lst, pointsAgo=1, updn='up'):
    lst = sorted(lst)
    if   updn == 'up': next_val = [x for x in lst if x > val][pointsAgo]
    elif updn == 'dn': next_val = [x for x in lst if x < val][pointsAgo]
    return next_val

next_hp = find_next_val(last_close, hp_lp, 1) 
next_lp = find_next_val(last_close, hp_lp, 1, 'dn')
print(f'{next_hp=}')
print(f'{next_lp=}')
print('------')

# 5. find the upper and lower bounds by seeing the max and min of the high and low points within a renge as determineed by a tolerance value which then computes the ART multiple of the range
def find_bounds(val, lst, tolerance=0.5):
    lst = sorted(lst)
    min_val = [x for x in lst if x < val][-1]
    max_val = [x for x in lst if x > val][0]
    rng = max_val - min_val
    ART = rng * tolerance
    return min_val - ART, max_val + ART

tolerance = 0.5
min_res, max_res = find_bounds(next_hp, hp_lp, tolerance=0.01)
print(f'{min_res=}')
print(f'{max_res=}')
print('------')

# log resistance and its bounds in a dictionary
resistance = {'lower': min_res, 'res': next_hp, 'upper': max_res}
print(f'{resistance=}')
print('------')

hplp

hp_lp=[87.14, 92.15, 91.51, 90.96, 93.89, 96.58, 96.58, 101.03, 85.92, 90.22, 90.09, 88.28, 89.36, 92.47, 93.83, 99.35, 95.34]
------
last_color='red'
last_close=np.float64(95.93)
------
next_hp=96.58
next_lp=87.14
------
min_res=95.29990000000001
max_res=99.39009999999999
------
resistance={'lower': 95.29990000000001, 'res': 96.58, 'upper': 99.39009999999999}
------


,HP,LP
date,,
2024-04-09 08:45:00,NaN,NaN
2024-04-09 09:00:00,NaN,NaN
2024-04-09 09:15:00,NaN,NaN
2024-04-09 09:30:00,NaN,NaN
2024-04-09 09:45:00,NaN,NaN
...,...,...
2024-04-12 17:15:00,NaN,NaN
2024-04-12 17:30:00,NaN,NaN
2024-04-12 17:45:00,NaN,NaN
